# Training an Azure AutoML model with Explanations
We are using credit risk modelling dataset from Kaggle for the task. In this project, we train an AutoML model using Azure AutoML and after selecting the model,
 we assess it's fairness and interpretablity.

#### Install Required Packages

In [ ]:
!pip install azureml-explain-model azureml-interpret azureml-sdk

#### Get Data

In [ ]:
from azureml.core import Workspace, Datastore, Dataset

# Get Workspace
ws = Workspace.from_config()

# Datastore Params
datastore_name = 'rohands'
storage_account_name = 'rohansa'
container_name ='rohan-blob'
# account_key = Add your account key for storage account

# Get Datastore otherwise create new datastore and register blob storage
if datastore_name in ws.datastores:
    blob_ds = Datastore.get(ws, datastore_name=datastore_name)
else:
    try:
        blob_ds = Datastore.register_azure_blob_container(
            workspace=ws,
            datastore_name=datastore_name,
            account_name=storage_account_name,
            #account_key = account_key
            container_name=container_name
        )
    except Exception as ex:
        print(ex)

# Set as Default Datastore
default_ds = ws.set_default_datastore(datastore_name)
default_ds = ws.get_default_datastore()

# Get Dataset otherise create new dataset and register
dataset_name = 'credit risk dataset'

if dataset_name not in ws.datasets:
    default_ds.upload_files(
        files=['./data/credit_risk_dataset.csv'],
        target_path='credit-risk-data/',
        overwrite=True,
        show_progress=True)
    
    tab_dataset = Dataset.Tabular.from_delimited_files(
        path=(default_ds,'credit-risk-data/*.csv'))
        
    try:
        tab_dataset = tab_dataset.register(workspace=ws, 
                                name=dataset_name,
                                description='credit risk data from Kaggle',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    tab_dataset = Dataset.get_by_name(ws, dataset_name)
    print('Dataset already registered.')

train_ds, test_ds = tab_dataset.random_split(percentage=0.7, seed=999)

##### Compute Target

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Compute params
compute_name = 'rohan-vm-cluster'
training_cluster = None

if compute_name in ws.compute_targets:
    training_cluster = ComputeTarget(ws, compute_name)
    print("Using existing cluster.")
else:
    try:
        compute_config = AmlCompute.provisioning_configuration(
            vm_size ='STANDARD_DS11_V2', 
            max_nodes=2 )
        training_cluster = ComputeTarget.create(ws, compute_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    print("Cluster created.")

#### Configure and Run AML

In [ ]:
from azureml.train.automl import AutoMLConfig
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

# Configuration
automl_config = AutoMLConfig(name='Automated Credit Risk Modelling',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='loan_status',
                             iterations=10,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )


# Run the Experiment
automl_experiment = Experiment(ws, 'azure-automl-fair-interpret-model')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion()

####  Get Best Run

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)

print('\nBest Model Definition:')
print(fitted_model)

print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
    
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

#### Explain Model using PFI Explainer and Get Global Feature Importance Values

In [ ]:
from interpret.ext.blackbox import PFIExplainer
import matplotlib.pyplot as plt

df = tab_dataset.to_pandas_dataframe()
features = list(df.columns).remove('loan_status')
classes = df.loan_status.unique()

X_train = train_ds.to_pandas_dataframe().drop('loan_status', axis=1)
y_train = train_ds.to_pandas_dataframe().loan_status

X_test = test_ds.to_pandas_dataframe().drop('loan_status', axis=1)
y_test = test_ds.to_pandas_dataframe().loan_status

pfi_explainer = PFIExplainer(model=fitted_model,
 features=features,
 classes=classes)

# PFIExplainer
global_explanation = pfi_explainer.explain_global(X_train, y_train)
global_feature_importance = global_explanation.get_feature_importance_dict()

# Figure
fig = plt.figure(figsize=(15,5))
top_n = 5 #len(global_feature_importance)
plt.bar(range(top_n), list(global_feature_importance.values())[:top_n], align='center')
plt.xticks(range(top_n), list(global_feature_importance.keys())[:top_n], rotation=15)
plt.tick_params(labelsize=18)

plt.title("Top {} Features".format(top_n), fontsize=24)
plt.xlabel("Features", fontsize=22)
plt.ylabel("Feature Importance", fontsize=22)
plt.show()

#### Register the Model

In [ ]:
# Register model
best_run.register_model(model_path='outputs/model.pkl', model_name='credit_risk_model',
    tags={'Training context':'Auto ML Credit Risk Model'},
    properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})